In [ ]:
import pandas as pd
import numpy as np
import os
import yaml
import csv
from icdmappings import Mapper

In [ ]:
# directory where the MIMIC-III data 'ADMISSIONS.csv', 'DIAGNOSES_ICD.csv', 'NOTEEVENTS.csv' are stored
mimic3_path = './MIMIC'

# directory where the filtered data will be stored
output_path = './MIMIC/filterd_data'

# directory where the ccs_definitions are stored
phenotype_definitions = './MIMIC/hcup_ccs_2015_definitions.yaml'

Goal:

process the admission table for readmission prediction;

filter out admissions based on admission types, patients' ages, and relation with diagnosis

In [ ]:
def dataframe_from_csv(path, header=0, index_col=0):
    return pd.read_csv(path, header=header, index_col=index_col)

def read_patients_table(mimic3_path):
    pats = dataframe_from_csv(os.path.join(mimic3_path, 'PATIENTS.csv'))
    pats = pats[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']]
    pats.DOB = pd.to_datetime(pats.DOB)
    pats.DOD = pd.to_datetime(pats.DOD)
    return pats


def read_admissions_table(mimic3_path):
    admits = dataframe_from_csv(os.path.join(mimic3_path, 'ADMISSIONS.csv'))
    #admits = admits[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'DIAGNOSIS']]
    admits.ADMITTIME = pd.to_datetime(admits.ADMITTIME, format='%Y-%m-%d %H:%M:%S', errors='coerce')
    admits.DISCHTIME = pd.to_datetime(admits.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    admits.DEATHTIME = pd.to_datetime(admits.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

    admits = admits.sort_values(['SUBJECT_ID','ADMITTIME'])
    admits = admits.reset_index(drop = True)

    admits['NEXT_ADMITTIME'] = admits.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
    admits['NEXT_ADMISSION_TYPE'] = admits.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

    rows = admits.NEXT_ADMISSION_TYPE == 'ELECTIVE'
    admits.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
    admits.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

    admits = admits.sort_values(['SUBJECT_ID','ADMITTIME'])

    admits[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = admits.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
    admits['DAYS_NEXT_ADMIT']=  (admits.NEXT_ADMITTIME - admits.DISCHTIME).dt.total_seconds()/(24*60*60)
    admits['OUTPUT_LABEL'] = (admits.DAYS_NEXT_ADMIT < 30).astype('int')
    return admits


def read_icustays_table(mimic3_path):
    stays = dataframe_from_csv(os.path.join(mimic3_path, 'ICUSTAYS.csv'))
    stays.INTIME = pd.to_datetime(stays.INTIME)
    stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
    return stays


def read_icd_diagnoses_table(mimic3_path):
    codes = dataframe_from_csv(os.path.join(mimic3_path, 'D_ICD_DIAGNOSES.csv'))
    codes = codes[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']]
    diagnoses = dataframe_from_csv(os.path.join(mimic3_path, 'DIAGNOSES_ICD.csv'))
    diagnoses = diagnoses.merge(codes, how='inner', left_on='ICD9_CODE', right_on='ICD9_CODE')
    diagnoses[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']] = diagnoses[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']].astype(int)
    return diagnoses

# def count_icd_codes(diagnoses, output_path=None):
#     codes = diagnoses[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']].drop_duplicates().set_index('ICD9_CODE')
#     codes['COUNT'] = diagnoses.groupby('ICD9_CODE')['ICUSTAY_ID'].count()
#     codes.COUNT = codes.COUNT.fillna(0).astype(int)
#     codes = codes[codes.COUNT > 0]
#     if output_path:
#         codes.to_csv(output_path, index_label='ICD9_CODE')
#     return codes.sort_values('COUNT', ascending=False).reset_index()


# def remove_icustays_with_transfers(stays):
#     stays = stays[(stays.FIRST_WARDID == stays.LAST_WARDID) & (stays.FIRST_CAREUNIT == stays.LAST_CAREUNIT)]
#     return stays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LAST_CAREUNIT', 'DBSOURCE', 'INTIME', 'OUTTIME', 'LOS']]


def merge_on_subject(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])


def merge_on_subject_admission(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

def diagnoses_groupby_adm(diagnoses): # column:[hadm_id, icd9_codes, ccs_codes]
  diagnoses_adm = diagnoses.groupby('HADM_ID')['ICD9_CODE'].apply(list).reset_index(name='ICD9_CODEs') #merge diagnosis codes of each admissions into a list
  mapper = Mapper()
  icd9list = list(diagnoses_adm.ICD9_CODEs)
  ccslist = []
  for codes in icd9list:
    ccslist.append(mapper.map(codes, source='icd9', target='ccs'))

  diagnoses_adm['CCS_CODES'] = ccslist
  return diagnoses_adm

def add_age_to_icustays(stays):
    stays['AGE'] = stays.apply(lambda e: (e['INTIME'].to_pydatetime()
                                          - e['DOB'].to_pydatetime()).total_seconds() / 3600.0 / 24.0 / 365.0,
                               axis=1)
    stays.loc[stays.AGE < 0, 'AGE'] = 90
    return stays

# def filter_admissions_on_nb_icustays(stays, min_nb_stays=1, max_nb_stays=1):
#     to_keep = stays.groupby('HADM_ID').count()[['ICUSTAY_ID']].reset_index()
#     to_keep = to_keep[(to_keep.ICUSTAY_ID >= min_nb_stays) & (to_keep.ICUSTAY_ID <= max_nb_stays)][['HADM_ID']]
#     stays = stays.merge(to_keep, how='inner', left_on='HADM_ID', right_on='HADM_ID')
#     return stays

def filter_icustays_on_age(stays, min_age=18, max_age=100):
    stays = stays[(stays.AGE >= min_age) & (stays.AGE <= max_age)]
    return stays


# def filter_diagnoses_on_stays(diagnoses, stays):
#     return diagnoses.merge(stays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID']].drop_duplicates(), how='inner',
#                            left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

def filter_diagnoses_on_admits(diagnoses, admits):
    return diagnoses.merge(admits[['SUBJECT_ID', 'HADM_ID']], how='inner',
                           left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

def filter_admits_on_ccsbenchmark(admits, phenotypes):
    return admits.merge(phenotypes[['HADM_ID']], how='inner',
                           left_on=['HADM_ID'], right_on=['HADM_ID'])

def add_hcup_ccs_2015_groups(diagnoses, definitions):
    def_map = {}
    for dx in definitions:
        for code in definitions[dx]['codes']:
            def_map[code] = (dx, definitions[dx]['use_in_benchmark'])
    diagnoses['HCUP_CCS_2015'] = diagnoses.ICD9_CODE.apply(lambda c: def_map[c][0] if c in def_map else None)
    diagnoses['USE_IN_BENCHMARK'] = diagnoses.ICD9_CODE.apply(lambda c: int(def_map[c][1]) if c in def_map else None)
    return diagnoses

# def add_inhospital_mortality_to_icustays(stays):
#     mortality = stays.DOD.notnull() & ((stays.ADMITTIME <= stays.DOD) & (stays.DISCHTIME >= stays.DOD))
#     mortality = mortality | (stays.DEATHTIME.notnull() & ((stays.ADMITTIME <= stays.DEATHTIME) & (stays.DISCHTIME >= stays.DEATHTIME)))
#     stays['MORTALITY'] = mortality.astype(int)
#     stays['MORTALITY_INHOSPITAL'] = stays['MORTALITY']
#     return stays


# def add_inunit_mortality_to_icustays(stays):
#     mortality = stays.DOD.notnull() & ((stays.INTIME <= stays.DOD) & (stays.OUTTIME >= stays.DOD))
#     mortality = mortality | (stays.DEATHTIME.notnull() & ((stays.INTIME <= stays.DEATHTIME) & (stays.OUTTIME >= stays.DEATHTIME)))
#     stays['MORTALITY_INUNIT'] = mortality.astype(int)
#     return stays

def make_phenotype_label_matrix(phenotypes):#, stays=None):
    phenotypes = phenotypes[['HADM_ID', 'HCUP_CCS_2015']].loc[phenotypes.USE_IN_BENCHMARK > 0].drop_duplicates()
    phenotypes['VALUE'] = 1
    phenotypes = phenotypes.pivot(index='HADM_ID', columns='HCUP_CCS_2015', values='VALUE')
    # if stays is not None:
    #     phenotypes = phenotypes.reindex(stays.ICUSTAY_ID.sort_values())
    return phenotypes.fillna(0).astype(int).sort_index(axis=0).sort_index(axis=1)

def remove_admissions_with_newborns_and_death(admits):
  admits = admits[admits['ADMISSION_TYPE']!='NEWBORN']
  admits = admits[admits.DEATHTIME.isnull()]
  admits['DURATION'] = (admits['DISCHTIME']-admits['ADMITTIME']).dt.total_seconds()/(24*60*60)

  admits = admits.reset_index(drop = True)
  return admits

In [ ]:
patients = read_patients_table(mimic3_path)
admits = read_admissions_table(mimic3_path)
stays = read_icustays_table(mimic3_path)

print('START:\n\tICUSTAY_IDs: {}\n\tHADM_IDs: {}\n\tSUBJECT_IDs: {}'.format(stays.ICUSTAY_ID.unique().shape[0],
          stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0]))

START:
	ICUSTAY_IDs: 61532
	HADM_IDs: 57786
	SUBJECT_IDs: 46476


In [ ]:
# remove newborn and death case in admmisions.csv
admits = remove_admissions_with_newborns_and_death(admits)

# double check...
if admits.DEATHTIME.isnull().unique()[0] != True:
  raise ValueError("There are still some death cases")

#admits.to_csv(os.path.join(output_path_ig, 'admissions.csv'))
print('HADM_IDs: {}'.format(admits.HADM_ID.unique().shape[0]))

HADM_IDs: 45321


In [ ]:
# obtain age information from 'stay' and gender informaton from 'patient'
stays = merge_on_subject(stays, patients)
stays_age = add_age_to_icustays(stays).drop_duplicates(subset=['HADM_ID'])
# keep 'age' in  (18,100)
stays_age = filter_icustays_on_age(stays_age)

In [ ]:
admits_age = pd.merge(admits[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION','DIAGNOSIS']],
                        stays_age[['SUBJECT_ID','HADM_ID','GENDER','AGE']],
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left').drop_duplicates(subset=['HADM_ID']) # 45321 admissions

In [ ]:
# merge information of diganosis codes and their description
diagnoses = read_icd_diagnoses_table(mimic3_path)
# group by diagnoses codes for each admission, store them in list, and map them to ccs codes
diagnoses_adm = diagnoses_groupby_adm(diagnoses) 

In [ ]:
admits_diagnosis = pd.merge(admits_age, diagnoses_adm, on=['HADM_ID'], how='left') #add diagnosis information to each admission
admits_diagnosis = admits_diagnosis[admits_diagnosis['ICD9_CODEs'].isnull() != True] # filter 10 admissions: 45302 left

In [ ]:
diagnosis_final = filter_diagnoses_on_admits(diagnoses, admits_diagnosis)
phenotypes = add_hcup_ccs_2015_groups(diagnosis_final, yaml.safe_load(open(phenotype_definitions, 'r')))
adm_phenotypes = make_phenotype_label_matrix(phenotypes)

In [ ]:
# add column HADM_ID based on index(hadm_id)
col = list(adm_phenotypes.columns)
hadm = list(adm_phenotypes.index)
adm_phenotypes_col_hadmid = pd.DataFrame(adm_phenotypes, columns=col, index=hadm)
adm_phenotypes_col_hadmid.reset_index(inplace=True)
adm_phenotypes_col_hadmid = adm_phenotypes_col_hadmid.rename(columns = {'index':'HADM_ID'})
adm_phenotypes_col_hadmid.to_csv(os.path.join(output_path, 'phenotypes_for_each_adm_final.csv'))

In [ ]:
admits_final = filter_admits_on_ccsbenchmark(admits_diagnosis, adm_phenotypes_col_hadmid)
admits_final.to_csv(os.path.join(output_path, 'admits_final.csv'))